# <center>Class 12: Time Series Regressions</center>

In [ ]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels as sm
import statsmodels.api as smapi

from stargazer.stargazer import Stargazer
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
# !pip install arch

In [ ]:
from arch.unitroot import PhillipsPerron, ADF

## Data - Equity Returns

In [ ]:
path = os.path.join(os.pardir, 'data', 'stock-prices-daily.csv') # this will produce a path with the right syntax for your operating system
path

In [ ]:
df_stocks = pd.read_csv(path)

In [ ]:
df_stocks

In [ ]:
df_stocks.info()

In [ ]:
df_stocks.date = pd.to_datetime(df_stocks.date, format= '%Y-%m-%d')

In [ ]:
df_stocks.info()

#### Feature engineering & EDA

In [ ]:
df_stocks["lnp_MSFT"] = np.log(df_stocks["p_MSFT"])

In [ ]:
df_stocks["lnp_SP500"] = np.log(df_stocks["p_SP500"])

In [ ]:
sns.lineplot(data = df_stocks, x = 'date', y = 'p_MSFT')
plt.ylabel('MSFT price in USD')
plt.grid(linestyle = 'dotted');

In [ ]:
g = sns.lineplot(data = df_stocks, x = 'date', y = 'p_MSFT')
g.set_yscale('log')
ylabels = ['{:,.0f}'.format(y) for y in g.get_yticks()]
g.set_yticklabels(ylabels)
plt.ylabel('MSFT price in USD')
plt.grid(linestyle = 'dotted');

In [ ]:
sns.lineplot(data = df_stocks, x = 'date', y = 'p_SP500')
plt.ylabel('S&P500 index level')
plt.grid(linestyle = 'dotted');

In [ ]:
sns.lineplot(data = df_stocks, x = 'date', y = 'p_SP500')
plt.yscale('log')
plt.ylabel('S&P500 index level')
plt.grid(linestyle = 'dotted');

**lagged values, log differences and percentage returns - daily frequency**

In [ ]:
df_stocks["l_p_MSFT"] = df_stocks["p_MSFT"].shift()
df_stocks["l_p_SP500"] = df_stocks["p_SP500"].shift()
df_stocks["d_p_MSFT"] = df_stocks["p_MSFT"] - df_stocks["l_p_MSFT"]
df_stocks["d_p_SP500"] = df_stocks["p_SP500"] - df_stocks["l_p_SP500"]
df_stocks["PctRetMSFT"] = df_stocks["d_p_MSFT"] / df_stocks["l_p_MSFT"] * 100
df_stocks["PctRetSP500"] = df_stocks["d_p_SP500"] / df_stocks["l_p_SP500"] * 100

df_stocks["d_lnp_MSFT"] = np.log(df_stocks["p_MSFT"]) - np.log(df_stocks["p_MSFT"].shift())
df_stocks["d_lnp_SP500"] = np.log(df_stocks["p_SP500"]) - np.log(df_stocks["p_SP500"].shift())

In [ ]:
df_stocks.dropna(inplace= True)

**lagged values, log differences and percentage returns - monthly frequency**

In [ ]:
df_stocks[["date", "year", "month", "p_SP500", "p_MSFT"]].groupby(df_stocks["date"].dt.to_period("M")).last()

In [ ]:
df_stocks_monthly = (
    df_stocks[["date", "year", "month", "p_SP500", "p_MSFT"]]
    .groupby(df_stocks["date"].dt.to_period("M"))
    .last()
    .reset_index(drop=True)
)

In [ ]:
df_stocks_monthly

In [ ]:
df_stocks_monthly["l_p_MSFT"] = df_stocks_monthly["p_MSFT"].shift()
df_stocks_monthly["l_p_SP500"] = df_stocks_monthly["p_SP500"].shift()
df_stocks_monthly["d_p_MSFT"] = df_stocks_monthly["p_MSFT"] - df_stocks_monthly["l_p_MSFT"]
df_stocks_monthly["d_p_SP500"] = df_stocks_monthly["p_SP500"] - df_stocks_monthly["l_p_SP500"]
df_stocks_monthly["PctRetMSFT"] = df_stocks_monthly["d_p_MSFT"] / df_stocks_monthly["l_p_MSFT"] * 100
df_stocks_monthly["PctRetSP500"] = (
    df_stocks_monthly["d_p_SP500"] / df_stocks_monthly["l_p_SP500"] * 100
)

df_stocks_monthly["d_lnp_MSFT"] = np.log(df_stocks_monthly["p_MSFT"]) - np.log(df_stocks_monthly["p_MSFT"].shift())
df_stocks_monthly["d_lnp_SP500"] = np.log(df_stocks_monthly["p_SP500"]) - np.log(df_stocks_monthly["p_SP500"].shift())

In [ ]:
df_stocks_monthly.dropna(inplace = True)

## Testing for Stationarity/Unit Root

#### Daily data

We cover two types of stationarity test:
- Augmented Dickey-Fuller (ADF)
- Phillips-Perron

In each case we test for:
- 'n': no constant term and no linear time trend
- 'c': constant term only
- 'ct': constant term and linear time trend

In all cases $H_0$: there is *unit root*, that is the *time series in NOT stationary*. $H_1$: it is stationary.

**ADF on the levels**

In [ ]:
print(ADF(df_stocks["p_MSFT"], lags = 32, trend="n").summary())

**Phillips-Perron on the levels**

<center>
    $y_i$ = $y_{i-1}$ + $\epsilon_i$
</center>

In [ ]:
print(PhillipsPerron(df_stocks["p_MSFT"], lags=32, test_type="rho", trend="n").summary())

**Note**: When using the PhillipsPerron test from the `arch` package in Python, setting `test_type='rho'` specifies that the test statistic should be based on the re-centered regression coefficient multiplied by the number of observations (nobs). This is different from the default `test_type='tau'`, which uses the t-statistic for the test.

In simpler terms, the rho test type focuses on the level of the autoregressive coefficient, while the tau test type focuses on the t-statistic of the coefficient. Both are used to test for the presence of a unit root, but they approach the calculation differently.

<center>
    $y_i$ = $\mu$ + $y_{i-1}$ + $\epsilon_i$
</center>

In [ ]:
print(PhillipsPerron(df_stocks["p_MSFT"], lags=32, test_type="rho", trend="c").summary())

<center>
    $y_i$ = $\mu$ + $y_{i-1}$ + $\beta$ * $t_i$ $ + \epsilon_i$
</center>

In [ ]:
print(PhillipsPerron(df_stocks["p_MSFT"], lags=32, test_type="rho", trend="ct").summary())

**Phillips-Perron on the log differences**

**Note**: These are *de-trended* values. 

In [ ]:
print(PhillipsPerron(df_stocks["d_lnp_MSFT"], lags=32, test_type="rho", trend="n").summary())

In [ ]:
print(PhillipsPerron(df_stocks["d_lnp_MSFT"], lags=32, test_type="rho", trend="c").summary())

In [ ]:
print(PhillipsPerron(df_stocks["d_lnp_MSFT"], lags=32, test_type="rho", trend="ct").summary())

#### Monthly Data

**levels**

In [ ]:
print(PhillipsPerron(df_stocks_monthly["p_MSFT"], lags=32, test_type="rho", trend="n").summary())

In [ ]:
print(PhillipsPerron(df_stocks_monthly["p_MSFT"], lags=32, test_type="rho", trend="c").summary())

In [ ]:
print(PhillipsPerron(df_stocks_monthly["p_MSFT"], lags=32, test_type="rho", trend="ct").summary())

**log differences**

In [ ]:
print(PhillipsPerron(df_stocks_monthly["d_lnp_MSFT"], lags=32, test_type="rho", trend="n").summary())

In [ ]:
print(PhillipsPerron(df_stocks_monthly["d_lnp_MSFT"], lags=32, test_type="rho", trend="c").summary())

In [ ]:
print(PhillipsPerron(df_stocks_monthly["d_lnp_MSFT"], lags=32, test_type="rho", trend="ct").summary())

## Behavior of Daily and Monthly Returns

In [ ]:
sns.lineplot(data = df_stocks, x = 'date', y = 'd_lnp_MSFT', size = 1, legend = None)
plt.hlines(0, df_stocks.date.min(), df_stocks.date.max(), color = 'k')
plt.grid(linestyle = 'dotted')
plt.ylabel('MSFT daily returns');

In [ ]:
sns.lineplot(data = df_stocks_monthly, x = 'date', y = 'd_lnp_MSFT', size = 1, legend = None)
plt.hlines(0, df_stocks.date.min(), df_stocks.date.max(), color = 'k')
plt.grid(linestyle = 'dotted')
plt.ylabel('MSFT monthly returns');

#### Percantage return statistics

In [ ]:
df_monthly_stats = df_stocks_monthly[['PctRetMSFT', 'PctRetSP500']].describe().T.round(2)
df_monthly_stats.index = ["Monthly returns on Microsoft (%)", "Monthly returns on S&P500 (%)"]
df_monthly_stats

**Question**: What do we measure using the standard deviation/variance of the returns?

#### Calculating market beta

In [ ]:
reg1 = smf.ols("PctRetMSFT ~ PctRetSP500", data = df_stocks_monthly).fit(cov_type="HC1")
reg2 = smf.ols("d_lnp_MSFT ~ d_lnp_SP500", data = df_stocks_monthly).fit(cov_type="HC1")
reg3 = smf.ols("PctRetMSFT ~ PctRetSP500", data = df_stocks).fit(cov_type="HC1")
reg4 = smf.ols("d_lnp_MSFT ~ d_lnp_SP500", data = df_stocks).fit(cov_type="HC1")

In [ ]:
print(reg1.summary())

In [ ]:
df_stocks_monthly['PtctMSFT_fitted'] = reg1.fittedvalues

In [ ]:
fig, ax = plt.subplots()

# data
ax.scatter(df_stocks_monthly.PctRetSP500, df_stocks_monthly.PctRetMSFT, s = 10, c = 'cornflowerblue')
ax.hlines(y = 0, xmin = df_stocks_monthly.PctRetSP500.min(), xmax = df_stocks_monthly.PctRetSP500.max(), color = 'cornflowerblue')
ax.plot(df_stocks_monthly.PctRetSP500, df_stocks_monthly.PctRetSP500, color = 'k', label = 'market beta')
ax.plot(df_stocks_monthly.PctRetSP500, df_stocks_monthly.PtctMSFT_fitted, 
        color = 'indianred', label = f'MSFT beta')

# aesthetics
plt.grid(linestyle = 'dotted')
plt.xlabel('S&P500 monthly returns')
plt.ylabel('MSFT monthly returns')
plt.legend(labelcolor = ['k', 'indianred'])
plt.text(x = -13, y = -22, s = f'MSFT beta: {reg1.params.PctRetSP500:.2f}', c = 'indianred');

In [ ]:
fig, ax = plt.subplots()
ax.hist(df_stocks.PctRetMSFT, bins = [x/2 - 10 for x in range(0,40, 1)], rwidth = 0.9, color = 'indianred', label = 'MSFT')
ax.hist(df_stocks.PctRetSP500, bins = [x/2 - 10 for x in range(0,40, 1)], rwidth = 0.9, color = 'white', edgecolor = 'k', alpha = 0.5, label = 'SP500')
plt.legend(labelcolor = ['indianred', 'grey'])
plt.xlabel('daily returns')
plt.ylabel('count')
plt.title('MSFT & SP500 risk profiles')
plt.grid(linestyle = ':');

**Question**: What does *fat tails* mean in finance?

**Market beta alternative measures**

In [ ]:
stargazer = Stargazer([reg1, reg2, reg3, reg4])
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "PctRetSP500": "S&P500 returns",
        "d_lnp_SP500": "S&P500 returns",
    }
)
stargazer.custom_columns(
    [
        "Monthly pct change",
        "Monthly log change",
        "Daily pct change",
        "Daily log change",
    ],
    [1, 1, 1, 1],
)
stargazer

## Data - Electricity & Climate

#### electricity

In [ ]:
path = os.path.join(os.pardir, 'data', 'electricity_resid_AZ.csv') # this will produce a path with the right syntax for your operating system
path

In [ ]:
df_electricity = pd.read_csv(path)

In [ ]:
df_electricity

In [ ]:
df_electricity["date"] = pd.to_datetime(df_electricity["MY"], format="%b-%y")

df_electricity["year"] = df_electricity["date"].dt.year
df_electricity["month"] = df_electricity["date"].dt.month
df_electricity["ym"] = (df_electricity["year"].astype(str).str.cat(df_electricity["month"].astype(str), sep="m"))

df_electricity = df_electricity[["Q", "date", "ym"]]

df_electricity["lnQ"] = df_electricity["Q"].map(np.log)

In [ ]:
df_electricity.head()

#### climate

In [ ]:
path = os.path.join(os.pardir, 'data', 'climate_Phoenix_AZ.csv') # this will produce a path with the right syntax for your operating system
path

In [ ]:
df_climate = pd.read_csv(path)

In [ ]:
df_climate

In [ ]:
df_climate["tempdate"] = pd.to_datetime(df_climate["DATE"], format="%Y-%m")

df_climate["year"] = df_climate["tempdate"].dt.year
df_climate["month"] = df_climate["tempdate"].dt.month
df_climate["ym"] = (
    df_climate["year"].astype(str).str.cat(df_climate["month"].astype(str), sep="m")
)

df_climate["ndays"] = 30
df_climate.loc[df_climate["month"].isin([1, 3, 5, 7, 8, 10, 12]), "ndays"] = 31
df_climate.loc[df_climate["month"] == 2, "ndays"] = 28

for x in ["CLDD", "HTDD", "DX70", "DX90"]:
    df_climate[x + "_avg"] = df_climate[x] / df_climate["ndays"]

df_climate = df_climate.drop(columns=["DATE", "tempdate", "STATION", "NAME"])

In [ ]:
df_climate.head()

In [ ]:
df_climate[["CLDD_avg", "HTDD_avg", "DX70_avg", "DX90_avg"]].describe().T.round(3)

#### merging into  single dataframe

In [ ]:
df_data = pd.merge(df_climate, df_electricity, on="ym", how="inner")

In [ ]:
df_data

In [ ]:
df_data = df_data[(df_data["year"] >= 2001) & (df_data["year"] <= 2017)].reset_index(drop=True)

In [ ]:
df_data.info()

In [ ]:
df_data.isna().sum().sum()

## Feature Engineering & EDA

In [ ]:
df_data[["Q", "lnQ", "CLDD_avg", "HTDD_avg"]].describe().T.round(3)

In [ ]:
df_data.plot( x = 'date', y = 'Q', color = 'k', 
             legend = False, grid = True,  );

In [ ]:
sns.lineplot(data = df_data, x = 'date', y = 'Q', legend = False, color = 'k')
plt.grid(linestyle = ':')
plt.ylabel('residential electricity consumption (GWh)');

In [ ]:
sns.lineplot(data = df_data, x = 'date', y = 'lnQ', legend = False, color = 'k')
plt.grid(linestyle = ':')
plt.ylabel('logged residential electricity consumption (ln GWh)');

In [ ]:
sns.lineplot(data = df_data, x = 'date', y = 'CLDD_avg', legend = False, color = 'k')
plt.grid(linestyle = ':')
plt.ylabel('average cooling degree days in Fahrenheit');

In [ ]:
sns.lineplot(data = df_data, x = 'date', y = 'HTDD_avg', legend = False, color = 'k')
plt.grid(linestyle = ':')
plt.ylim(0,30)
plt.ylabel('average heating degree days in Fahrenheit');

In [ ]:
sns.histplot(data = df_data, x = 'Q', bins = 30, shrink = 0.85, color = 'royalblue')
plt.xlabel('GWh')
plt.grid(linestyle = ':');

In [ ]:
df_data['CL_HT'] = df_data.CLDD_avg - df_data.HTDD_avg

In [ ]:
df_data['temp'] = df_data.CL_HT.map(lambda x: 'cold' if x < 0 else 'warm')

In [ ]:
sns.histplot(
    data = df_data, x = 'Q', 
    hue = 'temp', palette = ['royalblue', 'indianred'],
    bins = 30, shrink = 0.85)
plt.xlabel('GWh')
plt.grid(linestyle = ':');

In [ ]:
sns.histplot(
    data = df_data, x = 'lnQ', 
    hue = 'temp', palette = ['royalblue', 'indianred'],
    bins = 30, shrink = 0.85)
plt.xlabel('log GWh')
plt.grid(linestyle = ':');

In [ ]:
for x in ["lnQ", "CLDD_avg", "HTDD_avg", "DX90_avg"]:
    df_data["D" + x] = df_data[x] - df_data[x].shift()

In [ ]:
df_data.head()

In [ ]:
sns.histplot(
    data = df_data, x = 'DlnQ', 
    hue = 'temp', palette = ['royalblue', 'indianred'],
    bins = 30, shrink = 0.85)
plt.xlabel('logchange in GWh')
plt.grid(linestyle = ':');

In [ ]:
sns.boxplot(data = df_data, y = 'Q', x = 'month');
plt.xlabel('month')
plt.ylabel('GWh')
plt.title('Electricity consumption vs months')
plt.grid(linestyle = ':');

In [ ]:
sns.boxplot(data = df_data, y = 'CLDD', x = 'month');
plt.xlabel('month')
plt.ylabel('CLDD')
plt.title('Cooling degree days vs months')
plt.ylim(0, 1200)
plt.grid(linestyle = ':');

In [ ]:
sns.boxplot(data = df_data, y = 'HTDD', x = 'month');
plt.xlabel('month')
plt.ylabel('HTDD')
plt.ylim(0, 1200)
plt.title('Heating degree days vs months')
plt.grid(linestyle = ':');

#### What is the right functional form?

In [ ]:
sns.regplot(data = df_data, x="CLDD_avg", y="lnQ", 
            ci = None, line_kws= {'color': 'k'}, scatter_kws= {'s': 20},
            color = 'royalblue')
plt.grid(linestyle = ':')
plt.xlabel('average cooling degree days')
plt.ylabel('log electricity consuption (ln GWh)');

In [ ]:
sns.regplot(data = df_data, x="DCLDD_avg", y="DlnQ", 
            ci = None, line_kws= {'color': 'k'}, scatter_kws= {'s': 20},
            color = 'royalblue')
plt.grid(linestyle = ':')
plt.xlabel('average cooling degree days monthly change')
plt.ylabel('log electricity consumption monthly change (ln GWh)');

In [ ]:
sns.regplot(data = df_data, x="HTDD_avg", y="lnQ", 
            ci = None, line_kws= {'color': 'k'}, scatter_kws= {'s': 20},
            color = 'royalblue')
plt.grid(linestyle = ':')
plt.xlabel('average heating degree days')
plt.ylabel('log electricity consumption (ln GWh)');

In [ ]:
sns.regplot(data = df_data, x="DHTDD_avg", y="DlnQ", 
            ci = None, line_kws= {'color': 'k'}, scatter_kws= {'s': 20},
            color = 'royalblue')
plt.grid(linestyle = ':')
plt.xlabel('average heating degree days monthly change')
plt.ylabel('log electricity consumption monthly change (ln GWh)')
plt.text(x = -7.5, y = 0.3, s = 'linear\nregression', fontsize = 12);

In [ ]:
sns.regplot(data = df_data, x="DHTDD_avg", y="DlnQ", lowess = True,
            ci = None, line_kws= {'color': 'k'}, scatter_kws= {'s': 20},
            color = 'royalblue')
plt.grid(linestyle = ':')
plt.xlabel('average heating degree days monthly change')
plt.ylabel('log electricity consumption monthly change (ln GWh)')
plt.text(x = -7.5, y = 0.3, s = 'lowess\nregression', fontsize = 12);

In [ ]:
sns.regplot(data = df_data, x = 'CL_HT', y = 'Q', 
            lowess = True,
            # order = 2, 
            ci = None, 
            line_kws = {'color' : 'k'}, scatter_kws= {'s': 20})
plt.xlabel('cooling degree days - heating degree days')
plt.ylabel('GWh')
plt.grid(linestyle = ':');

In [ ]:
sns.regplot(data = df_data, x = 'CL_HT', y = 'Q', 
            # lowess = True,
            order = 2, 
            ci = None, 
            line_kws = {'color' : 'k'}, scatter_kws= {'s': 20})
plt.xlabel('cooling degree days - heating degree days')
plt.ylabel('GWh')
plt.grid(linestyle = ':');

#### Stationarity

**level**

In [ ]:
print(PhillipsPerron(df_data.lnQ, lags=12, test_type="rho", trend="n").summary())

**diff**

In [ ]:
print(PhillipsPerron(df_data[df_data.DlnQ.isna() == False].DlnQ, lags=12, test_type="rho", trend="n").summary())

## Linear Regression

**log consumption change vs change in HTDD & CLDD**

In [ ]:
reg1 = smf.ols("DlnQ ~ DCLDD_avg + DHTDD_avg", df_data).fit().get_robustcov_results(cov_type="HAC", maxlags=18)

In [ ]:
print(reg1.summary())

**log consumption change vs change in HTDD & CLDD + months as factors**

In [ ]:
reg2 = smf.ols("DlnQ ~ DCLDD_avg + DHTDD_avg + C(month)", df_data).fit().get_robustcov_results(cov_type="HAC", maxlags=18)

In [ ]:
print(reg2.summary())

**Question**: If months seem to be correlated with CLDD & HTDD, why do we have better results by adding the months as factors?

A good regression produces normally distributed residuals. Are our resudals follow a normal distribution (with $E({\epsilon_i})=0$)?

Visual inspection.

In [ ]:
plt.hist(reg2.resid, bins = 40, rwidth = 0.9)
plt.title('Regression 2 residual distribution');

Formal statistical test.

In [ ]:
from scipy.stats import kstest

stat, p = kstest(reg2.resid, 'norm')
print('Statistics=%.3f, p=%.3f' % (stat, p))

In [ ]:
kstest(reg2.resid, 'norm')

The $H_0$ of the `Kolmogorov-Smirnov Test` is that $x$ follows a normal distribution. Are our residuals normal?

**log consumption vs *level* of HTDD & CLDD + months as factors**

In [ ]:
reg3 = smf.ols("DlnQ ~ CLDD_avg + HTDD_avg + C(month)", df_data).fit().get_robustcov_results(cov_type="HAC", maxlags=18)

In [ ]:
print(reg3.summary())

In [ ]:
plt.hist(reg3.resid, bins = 40, rwidth = 0.9)
plt.title('Regression 2 residual distribution');

In [ ]:
stat, p = kstest(reg3.resid, 'norm')
print('Statistics=%.3f, p=%.3f' % (stat, p))

#### A digression into time series modelling: testing for autocorrelation

In [ ]:
from statsmodels.graphics import tsaplots

In [ ]:
#plot autocorrelation function
fig = tsaplots.plot_acf(reg3.resid, lags = 12)
plt.show()

`Autocorrelation` means that elements of a tine series are correlated with/dependent on their previous values. *Dependence means that these elements are not random*! If residuals from  time series are autocorrelated, that is they are not random, then we are missing some important patterns from our model. Remember: if we can identify the correct pattern, than all we have left - the residuals - should be completely random. 

Autocorrelation can be sometimes handled by using the lagged value(s) of our dependent variable amongst the independent variables. Sometimes this hepls, sometimes does not. 

The `acf` function of the `statsmodels.api.tsa` package calculates autocorrelation between variables and their lagged values.

In [ ]:
for i, v in enumerate(smapi.tsa.acf(reg3.resid)[0:13]):
    print('lag ' + str(i).rjust(2), str(f'{v:.3f}').rjust(7))

In [ ]:
reg4 = smf.ols("DlnQ ~ CLDD_avg + HTDD_avg + C(month) + DlnQ.shift(1)", df_data).fit().get_robustcov_results(cov_type="HAC", maxlags=18)

In [ ]:
print(reg4.summary())

In [ ]:
fig = tsaplots.plot_acf(reg4.resid, lags = 12)
plt.show()

#### Cumulative/long-run associations

In [ ]:
df_data["DDCLDD_avg"] = df_data["DCLDD_avg"] - df_data["DCLDD_avg"].shift()
df_data["DDHTDD_avg"] = df_data["DHTDD_avg"] - df_data["DHTDD_avg"].shift()

In [ ]:
reg5 = smf.ols(
    """DlnQ ~ 
        DCLDD_avg.shift(2) + DHTDD_avg.shift(2) + 
        DDCLDD_avg + DDHTDD_avg + 
        DDCLDD_avg.shift(1) + DDHTDD_avg.shift(1) + 
        C(month)""",
    df_data,
).fit()

In [ ]:
stargazer = Stargazer([reg5])
stargazer.covariate_order(["DCLDD_avg.shift(2)", "DHTDD_avg.shift(2)"])
stargazer.rename_covariates(
    {
        "DCLDD_avg.shift(2)": "ΔCD cumulative coeff",
        "DHTDD_avg.shift(2)": "ΔHD cumulative coeff",
    }
)
stargazer